# Tech Challenge Fase 3 - EDA
## Pós-graduação em Machine Learning
**Autor**: Alan Alves  
**Data**: 26/10/2025

In [ ]:
# Carrega Módulos
import pandas as pd
from pathlib import Path
from datetime import datetime
import sys
import numpy as np
from scipy import stats

In [2]:
# Configurações
PROJECT_ROOT = Path.cwd().parent
DATA_DIR = PROJECT_ROOT / "data"
print(f"Endereço raiz: {PROJECT_ROOT}")

Endereço raiz: c:\Users\avpalves\Downloads\temp\TechChallenge3


In [3]:
# Carrega módulos 
sys.path.append(str(Path.cwd().parent))
from src.utils import glimpse

### Inspeção Inicial

In [4]:
# Load data
airlines_path = DATA_DIR / "airlines.csv"
airport_path = DATA_DIR / "airports.csv"
flights_path = DATA_DIR / "flights.csv"

start_time = datetime.now()
airlines_df = pd.read_csv(airlines_path)
airports_df = pd.read_csv(airport_path)
flights_df = pd.read_csv(flights_path, dtype={
        'SCHEDULED_DEPARTURE': str,  
        'DEPARTURE_TIME': str
    })
end_time = datetime.now()

print(f"Dados carregados com sucesso em {((end_time-start_time).seconds)} seconds.")


C:\Users\avpalves\AppData\Local\Temp\ipykernel_6240\3460113801.py:9: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  flights_df = pd.read_csv(flights_path, dtype={


Dados carregados com sucesso em 28 seconds.


In [5]:
print("Airlines dataframe:")
glimpse(airlines_df)
print("\nAirports dataframe:")
glimpse(airports_df)
print("\nFlights dataframe:")
glimpse(flights_df)

Airlines dataframe:
Rows: 14
Columns: 2
           Null Count  Dtype   First Values
           ----------  -----   -------------
IATA_CODE  0           object  [UA, AA, US, F9, B6]
AIRLINE    0           object  [United Air Lines Inc., American Airlines Inc., US Airways Inc., Frontier Airlines Inc., JetBlue Airways]

Airports dataframe:
Rows: 322
Columns: 7
           Null Count  Dtype    First Values
           ----------  -----    -------------
IATA_CODE  0           object   [ABE, ABI, ABQ, ABR, ABY]
AIRPORT    0           object   [Lehigh Valley International Airport, Abilene Regional Airport, Albuquerque International Sunport, Aberdeen Regional Airport, Southwest Georgia Regional Airport]
CITY       0           object   [Allentown, Abilene, Albuquerque, Aberdeen, Albany]
STATE      0           object   [PA, TX, NM, SD, GA]
COUNTRY    0           object   [USA, USA, USA, USA, USA]
LATITUDE   3           float64  [40.65236, 32.41132, 35.04022, 45.44906, 31.53552]
LONGITUDE  3       

### Preparação Inicial dos Dados  

Os dados contém três tabelas:
* `flihgts_df` (voos): é a principal tabel a e contém informação detalhada de cada voo. 
* `airlines_df`: traz o nomes da empresas de aviações.
* `airports_df`  (aeroportos): traz informações sobre os aeroportos como por exemplo cidade, país etc.

**Obs.:** Existe varias colunas com falta de dados e algumas delas faltam dados para a maior parte das observações. 

O próximo passo é fazer a junção dessas três tabelas.

In [8]:
# --- Preparação Inicial - junção das três tabelas ---

# Função auxiliar para renomear colunas
def make_airport_columns_names(prefix):
    """Cria um dicionário para renomear colunas de aeroporto com um prefixo."""
    new_names_dict = {
        'AIRPORT': f'{prefix}_AIRPORT', 
        'CITY': f'{prefix}_CITY',
        'STATE': f'{prefix}_STATE', 
        'COUNTRY': f'{prefix}_COUNTRY', 
        'LATITUDE': f'{prefix}_LATITUDE',
        'LONGITUDE': f'{prefix}_LONGITUDE'
    }
    return new_names_dict

# Pula etapa de juntar tabelas caso o trabalha esteja sendo retomado e o processamento já havia sido realizado
df_out_path = PROJECT_ROOT / 'data/df.csv'
if df_out_path.exists():
    df = pd.read_csv(df_out_path, dtype={
        'SCHEDULED_DEPARTURE': str,  
        'DEPARTURE_TIME': str
    })
    
    # Libera memória
    del(flights_df)
    del(airlines_df)
    del(airports_df)
    
    print("Tabela com junções já realizada carregada do disco local:")
    glimpse(df)
else:
    # Junção com 'airlines_df' para obter o nome da companhia
    df = pd.merge(
        flights_df, 
        airlines_df, 
        left_on='AIRLINE', 
        right_on='IATA_CODE',
        suffixes=('_flight', '_airline'),
        how = 'left'
    )
    
    # Renomeia o nome da companhia e remove colunas redundantes
    df = df.rename(columns={'AIRLINE_airline': 'AIRLINE_NAME'})
    df = df.drop(columns=['AIRLINE_flight', 'IATA_CODE'])

    # Junção com 'airports_df' para dados de ORIGEM
    df = pd.merge(
        df, 
        airports_df, 
        left_on='ORIGIN_AIRPORT', 
        right_on='IATA_CODE',
        how = 'left'
    )
    
    # Renomeia colunas de origem e remove IATA_CODE redundante
    df = df.rename(columns=make_airport_columns_names('ORIGIN'))
    df = df.drop(columns=['IATA_CODE'])

    # Junção com 'airports_df' para dados de DESTINO
    df = pd.merge(
        df, 
        airports_df, 
        left_on='DESTINATION_AIRPORT', 
        right_on='IATA_CODE',
        how = 'left'
    )
    
    # Renomeia colunas de destino e remove IATA_CODE redundante
    df = df.rename(columns=make_airport_columns_names('DEST'))
    df = df.drop(columns=['IATA_CODE'])

    # Salva tabela
    df.to_csv(df_out_path, index=False)

    # Libera memória
    del(flights_df)
    del(airlines_df)
    del(airports_df)

    print("Associações realizada e tabela salva com sucesso para o caminho selecionado. ")
    df.info()

C:\Users\avpalves\AppData\Local\Temp\ipykernel_6240\470493695.py:19: DtypeWarning: Columns (6,7,31,32,33,34,37,38,39,40) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(df_out_path, dtype={


NameError: name 'flights_df' is not defined

### Formatando Colunas de Tempo 
As colunas `SCHEDULED_DEPARTURE` e `DEPARTURE_TIME` estão no formato HHMM e para ser util é necessário separar em duas colunas contendo a hora e o minuto e converte-las para número. Outras colunas também apresentam esse mesmo formato, mas será alterada apeas se avaliarmos ser útil para análise mais a frente. 

In [10]:
# Obtem a data em uma única coluna
df['DATE'] = pd.to_datetime(df[['YEAR', 'MONTH', 'DAY']])

# Obtem hora e minutos em coluna separada
df['SCHEDULED_DEPARTURE_HOUR'] = df['SCHEDULED_DEPARTURE'].str[:2]
df['SCHEDULED_DEPARTURE_MIN'] = df['SCHEDULED_DEPARTURE'].str[2:4]
df['DEPARTURE_HOUR'] = df['DEPARTURE_TIME'].str[:2]
df['DEPARTURE_MIN'] = df['DEPARTURE_TIME'].str[2:4]

# Converte para número
df['SCHEDULED_DEPARTURE_HOUR'] = pd.to_numeric(df['SCHEDULED_DEPARTURE_HOUR'], errors='coerce').astype('Int64')
df['SCHEDULED_DEPARTURE_MIN'] = pd.to_numeric(df['SCHEDULED_DEPARTURE_MIN'], errors='coerce').astype('Int64')
df['DEPARTURE_HOUR'] = pd.to_numeric(df['DEPARTURE_HOUR'], errors='coerce').astype('Int64')
df['DEPARTURE_MIN'] = pd.to_numeric(df['DEPARTURE_MIN'], errors='coerce').astype('Int64')



### Sumário Descritivo das Colunas

In [14]:
round(df.describe().T,2)

,count,mean,min,25%,50%,75%,max,std
YEAR,5819079.0,2015.0,2015.0,2015.0,2015.0,2015.0,2015.0,0.0
MONTH,5819079.0,6.524085,1.0,4.0,7.0,9.0,12.0,3.405137
DAY,5819079.0,15.704594,1.0,8.0,16.0,23.0,31.0,8.783425
DAY_OF_WEEK,5819079.0,3.926941,1.0,2.0,4.0,6.0,7.0,1.988845
FLIGHT_NUMBER,5819079.0,2173.092742,1.0,730.0,1690.0,3230.0,9855.0,1757.063999
DEPARTURE_DELAY,5732926.0,9.370158,-82.0,-5.0,-2.0,7.0,1988.0,37.080942
TAXI_OUT,5730032.0,16.071662,1.0,11.0,14.0,19.0,225.0,8.895574
WHEELS_OFF,5730032.0,1357.170841,1.0,935.0,1343.0,1754.0,2400.0,498.009356
SCHEDULED_TIME,5819073.0,141.685892,18.0,85.0,123.0,173.0,718.0,75.210582
ELAPSED_TIME,5714008.0,137.006189,14.0,82.0,118.0,168.0,766.0,74.211072


### Análise de Estatística Descritiva e Tratamento de Dados Ausentes

1. **Dataset**:
    * 5,8M de registros, suficiente para treinar qualquer modelo de Machine Learning.
    * Escopo Temporal: Todos os registros são do ano de 2015 (min e max da coluna year igual a 2015). Sazonalidade ao longo do ano pode ser capturada, mas não será possível calcular sozonalidade ou mudança anual. **Devido não ser possível avaliar a tendência ano-a-ano, cuidados devem ser tomados ao colocar o modelo em produção**, pois a realidade atual pode ser diferente de 10 anos atrás. 

2. **Variaveis resposta: DEPARTURE_DELAY e ARRIVAL_DELAY**

    * *ARRIVAL_DELAY*:
        * voos chegam um pouco atrasados - média de 4.4 minutos. 
        * Metade dos voos chegam pelo menos 5 minutos adiantados - mediana de -5min
        * Atrasos extremos (máxima de 1971 min) fazem a média de voos serem atrasados em 4.4min. Contudo, mediana -5min, indica distribuição assimétrica com a maioria dos voos *não atrasando na chegada* e poucos outros voos com atrasos curtos e *outros em menores quantidades, com atrasos extremos*
        * O devio padrão é quase 9 vezes maior que a média, indicando dados espalhados, dificeis de serem previstos. O projeto de classificação (ATRASO vs NÃO ATRASO), terá maior probabilidade de sucesso.

    * *DEPARTURE_DELAY*:
        * Média de 9.37 minutos, e mediana de -2 minutos. 50% dos voos saem no horário ou adiantados, mas a menor quantidade de voos que saem atrasados, puxa a média para cima, implicando em uma distribuição assimétrica. 

    * **Implicação para Modelagem**:
        * *Variável alvo*: talvez, mais importante que saber se o voo irá sair atrasado é saber se chegará atrasado. Como a média da variável `DEPARTURE_DELARY`é mais que o dobro da variavel `ARRIVAL_DELAY` é provavel que nem todo voo que sai atrasado chega atrasado. Portanto, **a variavel alvo a ser trabalhada é `ARRIVAL_DELAY`**. 
        
        * *ATRASADO*: para muitas aplicações, atrasos de 5min é suficiente para desencadear uma série de situações indesejadas. Tendo isso em mente, uma nova váriavel, `IS_DELAYED` será criada. Essa nova variável **indica se um voo chegou atrasado em seu destino, por 5min ou mais**.
        
        * *Classe desbalanceada*: definir o limit para atraso como 5min, significa puxar mais à direita (destro da distribuição) o limite, indicando um desbalaceamento entre as classes, que impactará diretamente na modelagem.

3. **Dados Ausentes**
    * Voos Concluídos: As colunas ARRIVAL_DELAY e ELAPSED_TIME têm 5.714.008 registros, enquanto os dados tem 5.819.079 registros. A diferença (~104k) provavelmente corresponde aos voos CANCELLED (cancelado - 1.5% do total, ~89k) e DIVERTED (desviado, 0.26% do total, ~15k). A coluna `MEAN` indica a fração de voos de cancelados e desviados. Portanto a quantidade de observações disponível para modelagem de voos atrasado é **5.714.008**. **Tratamento**: excluir observações com dados ausentes para `ELAPSED_TIME`. O mesmo se aplica a outras colunas relacionadas, como por exemplo `TAXI_IN`, `WHEELS_ON` e `AIR_TIME`.

    * Colunas de atraso: colunas que finalizam com o sufixo **DELAY** são colunas que indicam razões para atrasos. Apenas 1.063.439, são preenchidas. Hipótese: Elas só são preenchidas quando há atraso . **Tratamento**: Preencher com 0, assumindo que NaN significa "0 minutos de atraso" daquele tipo.

    * *Latitude/Longitude*: Quase 500k voos estão sem dados de ORIGIN_LATITUDE ou DEST_LATITUDE. Hipótese: é possível que o arquivo airports.csv não contenha todos os IATA_CODEs presentes nos voos, talvez para aeroportos com menores ou regionais, talvez aeroportos menores ou regionais. **Tratamento**: para análises geográficas ou que dependem da localização, essas linhas serão removidas. Para modelagem, novas variáveis será criadas a partir dela, como por exemplo, `REGIONAL_AIRPORT`, sendo verdadeiro para dados ausentas.

4. **Variáveis Explicatórias**
    * *TAXI_OUT e TAXI_IN*:  O tempo médio de TAXI_OUT (16.07 min) é mais que o dobro do TAXI_IN (7.43 min). Possivelmente o tempo de *taxiamento* para decolagem influencia mais os atrasos (assumindo que a coluna ARRIVAL_TIME é indica o tempo após o TAXI_IN) que o *taxiamento* pós pouso. A variável `TAXI_IN`, não fará parte da próxima etapa de EDA e nem da modelagem, pois é evento que ocorre imediatamente antes da conlusão do voo e seu efeito prático é nulo. 

    * *LATE_AIRCRAFT_DELAY* tem a maior média (23.47 min - calculado apenas entre os voos atrasados) entre os tipos de atraso. Isso sugere que um voo atrasado é um forte preditor de que o próximo voo daquela aeronave também atrasará.

    * *SECURITY_DELAY* (média 0.076 min) é praticamente irrelevante na maioria dos casos. 75% dos voos (75% percentil) com atraso detalhado tiveram 0 minutos de atraso de segurança.

    * *Horários*: As colunas de `HOUR` e `MIN` criadas estão bem distribuídas (média e mediana centralizadas). Podem ser utilizada com segurança para verificar se existe padrão de atraso a depender da hora do dia, por exemplo.

    * *DISTANCE*: Os voos variam de muito curtos (21 milhas) a muito longos (4983 milhas), com a maioria sendo de curta a média distância (mediana de 647 milhas).

## Modelagem supervisonada - Classificação e Problemática

Para efeitos práticos, o modelo servirá como solução para o problema de **perda de conexões devido à atraso nos voos**.
Algumas conexões são curtas (da ordem de 25 a 30) e atraso de 5min pode implicar na perda da conexão por um passageiro, portanto, ser capaz de prever se um voo irá atrasar por 5min ou mais, com pelo menos 30min de atecedencia, para ajudar na **gestão proativa de conexões de passageiros**, como por exemplo reservando outro voo para o passageiro e tomando os medidas que ameniza o decontentamento do cliente, diminuindo o impacto negativo no negócio. 

**Objetivo**: prever se um voo irá se atrasar por 5min ou mais, com 30min de atecedencia à chegada agendada do voo. 

In [ ]:
df = df[df['CANCELLED'] == 0 & df['DIVERTED'] == 0]

In [ ]:
# Insights

In [21]:
stats.percentileofscore(df['DEPARTURE_DELAY'], 0, kind='weak', nan_policy='omit')

np.float64(62.922633224290706)